In [1]:
! pip install tensorflow numpy

  Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached wrapt-1.17.2-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer

<h2>Creating Custom SelfAttention Layer</h2>

In [4]:
class SelfAttention(Layer):
    def __init__(self, dim):
        super(SelfAttention, self).__init__()
        self.embed_dim = dim
        self.query = tf.keras.layers.Dense(dim)
        self.key = tf.keras.layers.Dense(dim)
        self.value = tf.keras.layers.Dense(dim)
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, inputs):
        Q = self.query(inputs)
        K = self.key(inputs)
        V = self.value(inputs)

        qk = tf.matmul(Q, K, transpose_b=True)
        sqrt = tf.math.sqrt(tf.cast(self.embed_dim, tf.float32))
        inner = qk/sqrt

        attention_weight = self.softmax(inner)
        result = tf.matmul(attention_weight, V)

        return result

In [7]:
x = tf.random.normal((2, 5, 64))
attention = SelfAttention(dim = 64)
attention_val = attention(x)
attention_val.shape

TensorShape([2, 5, 64])

<h2>Using predefined Multihead Attention layer</h2>

In [10]:
from tensorflow.keras.layers import MultiHeadAttention, Input
from tensorflow.keras.models import Model

In [11]:
input_tensor = Input(shape=(5,64))

In [14]:
multihead_output = MultiHeadAttention(num_heads=4, key_dim=16)(input_tensor, input_tensor)

In [15]:
model = Model(inputs=input_tensor, outputs=multihead_output)

In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5, 64)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 64)     │     16,640 │ input_layer[0][0… │
│ (MultiHeadAttentio… │                   │            │ input_layer[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,640 (65.00 KB)

 Trainable params: 16,640 (65.00 KB)

 Non-trainable params: 0 (0.00 B)